In [1]:
from pymongo import MongoClient

# Connect to mongodb and create a new database
client = MongoClient("mongodb://localhost:27017/")
database = client["MedGraph"]

# Base pubmed url: https://pubmed.ncbi.nlm.nih.gov/ + paper_id

In [5]:
import pymongo, networkx

def build_graph_from_meshterms(collection: pymongo.collection.Collection):
    graph = networkx.Graph()
    nodes_list = []
    edges_list = []

    for document in collection.find():
        current_pmid = document["_id"]
        graph.add_node(current_pmid)
        
        for document2 in collection.find():
            count = 0
            if current_pmid != document2["_id"]:
                for key in document["mesh_terms"]:
                    if key in document2["mesh_terms"]: count += 1
            
            if count > 0:
                graph.add_edge(current_pmid, document2["_id"], weight = count)    

    return graph

# execution time: 1min 29.4s
# execution time: 51.2s

In [6]:
collection = database["Dataset2000Entries"]
gx = build_graph_from_meshterms(collection)